In [2]:
import ipywidgets as widgets
import pandas as pd
from IPython.display import display, HTML

from jobsearch.utils import validate_jsonschema
from jobsearch.database import fetch_data_from_postgresql
import pandas as pd

LOCAL_SAVE_PATH = '../../data/fine-tuning_data/salary_ft_data_2.csv'


In [8]:
data = pd.read_csv('../../data/fine-tuning_data/poste_RAW_ft_data.csv')

In [9]:
df = data.copy()
df.head()

,id,description,poste,jsonschema_validity
0,50166,Type de poste : CDI Salaire : En fonction de l...,"{\n ""poste"": {\n ""type_contrat"": ""CDI"",\n ...","'{\n ""poste"": {\n ""type"": ""object"",\n ""..."
1,50216,Le Groupe DEMETER est un cabinet de recrutemen...,"{\n ""poste"": {\n ""type_contrat"": ""CDI"",\n ...","'{\n ""poste"": {\n ""type"": ""object"",\n ""..."
2,50320,Critères de l'offre\n• Data Engineer (H/F)\n• ...,"{\n ""poste"": {\n ""type_contrat"": ""CDI"",\n ...","'{\n ""poste"": {\n ""type"": ""object"",\n ""..."
3,50367,Connecting talent to the Energy industry. Cons...,"{\n ""poste"": {\n ""type_contrat"": ""freelanc...","'{\n ""poste"": {\n ""type"": ""object"",\n ""..."
4,50393,Company Description Who are we ? MANGOPAY’s vi...,"{\n ""poste"": {\n ""type_contrat"": ""CDI"",\n ...","'{\n ""poste"": {\n ""type"": ""object"",\n ""..."


In [10]:
# Apply custom CSS to the notebook for widget styling
display(HTML("""
<style>
    .widget-textarea { width: 100% !important; }
    .centered { display: flex; justify-content: center; }
    .widget-label { min-width: 20ex; }
    .hbox { align-items: center; }
</style>
"""))


current_index = 0  # Initialize a variable to track the current record index

# Function to save changes to the dataframe
def save_changes(b):
    # Update the dataframe with changes from the text areas
    df.at[current_index, 'description'] = description_text.value
    df.at[current_index, 'poste'] = salary_text.value
    
    df.to_csv(LOCAL_SAVE_PATH, index=False)
    
    with output:
        output.clear_output()
        print("Changes saved.")

# Function to load a new record into the text areas
def load_record(index):
    global current_index
    
    current_index = index
    description_text.value = df.at[current_index, 'description']
    salary_text.value = df.at[current_index, 'poste']

# Update record index and load a new record on button click
def update_record(change):
    global current_index
    
    # Save the current record before moving to the next or previous one
    save_changes(None)  # Passing None because we don't use the button argument
    
    # Move to the next or previous record if possible
    if change == 'next' and current_index < len(df) - 1:
        current_index += 1
    elif change == 'prev' and current_index > 0:
        current_index -= 1
    record_dropdown.value = current_index  # This will update the dropdown value
    load_record(current_index)  # Explicitly call the load record

# Create text areas for the description and salary with a larger fixed height
description_text = widgets.Textarea(value='', layout=widgets.Layout(width='auto', height='1000px')) # Adjust the height as needed
salary_text = widgets.Textarea(value='', layout=widgets.Layout(width='auto', height='1000px')) # Adjust the height as needed

# Create a button to save changes and assign the function to it
save_button = widgets.Button(description="Save Changes")
save_button.on_click(save_changes)

# Create navigation buttons for previous and next records
prev_button = widgets.Button(description='Previous')
next_button = widgets.Button(description='Next')
prev_button.on_click(lambda b: update_record('prev'))
next_button.on_click(lambda b: update_record('next'))

# Create a dropdown to navigate records
record_dropdown = widgets.Dropdown(options=list(df.index), value=0)
record_dropdown.observe(lambda change: load_record(change['new']), names='value')

# Create an output widget to display messages
output = widgets.Output()

# Arrange the widgets with proper styling
navigation_buttons = widgets.HBox([prev_button, next_button], layout=widgets.Layout(justify_content='center'))
selection_and_save = widgets.HBox([record_dropdown, save_button], layout=widgets.Layout(justify_content='center'))
display_widgets = widgets.HBox([description_text, salary_text], layout=widgets.Layout(justify_content='center'))
messages = widgets.HBox([output], layout=widgets.Layout(justify_content='center'))

# Load the first record by default
load_record(current_index)

# Combine all the widgets into a VBox to display them vertically
widgets.VBox([selection_and_save, display_widgets, navigation_buttons, messages])
            